In [1]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
def my_huber_loss_1(y_true, y_pred):
    threshold = 1.
    error = y_true - y_pred
    is_small_error = tf.abs(error) <= threshold
    small_error_loss = tf.square(error) / 2
    big_error_loss = threshold * (tf.abs(error) - threshold / 2)
    
    loss = tf.where(is_small_error, small_error_loss, big_error_loss)
    
    col1 = tf.transpose(tf.expand_dims(loss[:, 1] * 0, axis = 0))
    
    return tf.concat([loss[:, :1], col1], axis = 1)

def my_huber_loss(y_true, y_pred):
    threshold = 1.
    error = y_true - y_pred
    is_small_error = tf.abs(error) <= threshold
    small_error_loss = tf.square(error) / 2
    big_error_loss = threshold * (tf.abs(error) - threshold / 2)
    
    loss = tf.where(is_small_error, small_error_loss, big_error_loss)
    
    return loss

In [3]:
y_true = np.array([[0.1, 0.2], 
                   [0.3, 0.4]], np.float32)
y_pred = np.array([[0.2, 0.25], 
                   [0.29, 0.33]], np.float32)

test = my_huber_loss(y_true, y_pred)

2022-09-04 02:25:29.111786: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
test

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[5.0000004e-03, 1.2499999e-03],
       [5.0000202e-05, 2.4499996e-03]], dtype=float32)>

In [5]:
col1 = tf.transpose(tf.expand_dims(test[:, 1] * 2, axis = 0))

tf.concat([test[:, :1], col1], axis = 1)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[5.0000004e-03, 2.4999997e-03],
       [5.0000202e-05, 4.8999991e-03]], dtype=float32)>

In [6]:
xs = np.array([[-1.0, 0.0], [1.0, 2.0], [3.0, 4.0]], dtype=float)
# labels
ys = np.array([[-3.0, -1.0], [1.0, 3.0], [5.0, 7.0]], dtype=float)

In [7]:
my_huber_loss(y_true, y_pred)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[5.0000004e-03, 1.2499999e-03],
       [5.0000202e-05, 2.4499996e-03]], dtype=float32)>

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=6, input_shape=[2,]),
    tf.keras.layers.Dense(units=2, input_shape=[6,]),
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 18        
                                                                 
 dense_1 (Dense)             (None, 2)                 14        
                                                                 
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________


In [9]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(units=6, input_shape=[2,]),
#     tf.keras.layers.Dense(units=2, input_shape=[6,]),
# ])

input = tf.keras.layers.Input(shape=(2), name="input")

input_layer = tf.keras.layers.Dense(units=6)(input)
fc2 = tf.keras.layers.Dense(units=2,  name='final_layer_1')(input_layer)
fc3 = tf.keras.layers.Dense(units=2,  name='final_layer_2')(input_layer)

# model.compile(optimizer='sgd', loss=[my_huber_loss])

In [10]:
model = tf.keras.models.Model(inputs=input,  outputs=[fc2, fc3])

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 2)]          0           []                               
                                                                                                  
 dense_2 (Dense)                (None, 6)            18          ['input[0][0]']                  
                                                                                                  
 final_layer_1 (Dense)          (None, 2)            14          ['dense_2[0][0]']                
                                                                                                  
 final_layer_2 (Dense)          (None, 2)            14          ['dense_2[0][0]']                
                                                                                              

In [12]:
pred = model.predict(xs) 

1/1 [==============================] - 0s 112ms/step


In [13]:
np.sum(my_huber_loss_1( ys , pred[0])) / 6

1.954270839691162

In [14]:
my_huber_loss_1( ys , pred[1])

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[3.5916357 , 0.        ],
       [1.37892771, 0.        ],
       [0.0557048 , 0.        ]])>

In [15]:
np.sum(my_huber_loss( ys , pred[1])) / 6

2.868728295940175

In [16]:
model.compile(optimizer='sgd', loss=[my_huber_loss_1, my_huber_loss])

In [17]:
model.fit(xs, [ys, ys], epochs=50, verbose=1)

Epoch 1/50
1/1 [==============================] - 1s 832ms/step - loss: 4.8230 - final_layer_1_loss: 1.9543 - final_layer_2_loss: 2.8687
Epoch 2/50
1/1 [==============================] - 0s 5ms/step - loss: 4.7115 - final_layer_1_loss: 1.9087 - final_layer_2_loss: 2.8027
Epoch 3/50
1/1 [==============================] - 0s 5ms/step - loss: 4.6059 - final_layer_1_loss: 1.8651 - final_layer_2_loss: 2.7408
Epoch 4/50
1/1 [==============================] - 0s 4ms/step - loss: 4.5049 - final_layer_1_loss: 1.8230 - final_layer_2_loss: 2.6818
Epoch 5/50
1/1 [==============================] - 0s 4ms/step - loss: 4.4073 - final_layer_1_loss: 1.7823 - final_layer_2_loss: 2.6249
Epoch 6/50
1/1 [==============================] - 0s 4ms/step - loss: 4.3123 - final_layer_1_loss: 1.7428 - final_layer_2_loss: 2.5695
Epoch 7/50
1/1 [==============================] - 0s 4ms/step - loss: 4.2192 - final_layer_1_loss: 1.7042 - final_layer_2_loss: 2.5150
Epoch 8/50
1/1 [==============================] - 0s 

In [18]:
model.predict(xs)

1/1 [==============================] - 0s 51ms/step


[array([[-0.9489418 , -0.16446897],
        [ 1.5946609 , -0.4316575 ],
        [ 4.1382637 , -0.6988462 ]], dtype=float32),
 array([[ 0.5313915, -0.9613741],
        [ 2.505542 ,  3.0836008],
        [ 4.479692 ,  7.128576 ]], dtype=float32)]